In [37]:
bronze_df = spark.sql("SELECT * FROM bronze_table_lh")


StatementMeta(, f92d965a-f3f3-4f06-b18e-8455a230491e, 40, Finished, Available, Finished, False)

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window


spark = SparkSession.builder.appName("SilverLayerBatch").getOrCreate()


bronze_df = spark.sql("SELECT * FROM bronze_table_lh")





bronze_df = bronze_df.withColumn("event_epoch", F.col("timestamp").cast("long"))

# Define windows using epoch
w_1min = Window.partitionBy("user_id").orderBy("event_epoch") \
    .rangeBetween(-60, 0)   # last 60 seconds

w_24h = Window.partitionBy("user_id").orderBy("event_epoch") \
    .rangeBetween(-86400, 0)   # last 24 hours


w_prev_city = Window.partitionBy("user_id").orderBy("timestamp")
w_merchant = Window.partitionBy("merchant_id")



silver_df = (
    bronze_df
    .withColumn("txn_count_1min_per_user", F.count("*").over(w_1min))
    .withColumn("avg_amount_24h", F.round(F.avg("amount").over(w_24h), 2))   # rounded to 2 decimals
    .withColumn("amount_deviation", F.round(F.col("amount") - F.col("avg_amount_24h"), 2))  # rounded to 2 decimals
    .withColumn("prev_city", F.lag("city").over(w_prev_city))
    .withColumn("geo_change_flag", F.when(F.col("city") != F.col("prev_city"), 1).otherwise(0))
    .drop("prev_city")
    .withColumn("merchant_risk_score",
                F.sum(F.col("is_fraud").cast("int")).over(w_merchant) /
                F.count("*").over(w_merchant))
)




silver_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver_table_lh")

print("Silver table written successfully as managed table: silver_table_lh")


display(spark.sql("SELECT * FROM silver_table_lh LIMIT 10"))


StatementMeta(, f92d965a-f3f3-4f06-b18e-8455a230491e, 44, Finished, Available, Finished, False)

Silver table written successfully as managed table: silver_table_lh


SynapseWidget(Synapse.DataFrame, 61a17ff7-2146-45bb-832c-10c78a989ba2)